# Algorithms for Data Science

## Computing Moments of a Stream

### 1. Preliminaries 

The objective of this lab is to implement the Alon-Matias-Szegedy approach to estimate the second moment of the stream in which $N$ distinct items from $0$ to $N-1$ appear.

In [5]:
import random

#parameters
N = 256 #N distinct values between 0 and N-1
stream_size = 10000

### 2. Alon-Matias-Szegedy for Second Moments

We implement here the AMS approach when the stream size is known:
1. We choose a number $t$ between $0$ and $\text{stream\_size}-1$ from which the counts are kept
2. When the stream is at timestamp $t$, we initialize $\text{v}=S(t)$ and $c=1$
3. Whenever we encounter $v$ in the stream, we increment $c$ by $1$

At the end of the stream, we output the estimator $\text{stream\_size}\times(2c-1)$

This can be easily extended to an arbirary number of counts, by generating $k$ different timestamps and keeping arrays of $v$ and $c$.

In [26]:
#initialize values and counts
v = []
c = []
#keeping the true counts 
counts = {}
#choosing k timestamps
k = 10
t = []
for _ in range(k):
  t.append(random.randrange(stream_size))
  v.append(-1)
  c.append(0)

for i in range(stream_size):
  #take a random value between 0 and N-1
  s = random.randrange(N)
  #AMS approach
  for j in range(k):
    if i==t[j]: #chosen timestamp
      v[j] = s
      c[j] = 1
    elif i>t[j] and s==v[j]: #after timestamp
      c[j] += 1
  #true counts (only for evaluation!)
  if s not in counts:
    counts[s] = 0
  counts[s] = counts[s]+1

#true 2nd moment
true = 0
for x in counts.keys():
  true += counts[x]*counts[x]

#2nd moment estimator
est = 0
for x in range(k):
  est += 2*c[x]-1
est = int((stream_size/k)*est)

print('Estimation of 2nd moment: %d'%est)
print('True second moment: %d'%true)

Estimation of 2nd moment: 388000
True second moment: 399904


### 3. **TASK** AMS for Infinite Streams

Implement the case when the estimator does not know the size of the stream.

In this case, instead of generating $k$ timestamps, we proceed to use _Reservoir Sampling_ as explained in the lecture:
1. initialize $v$ and $c$ with the corresponding values in the first $k$ items in the stream $S$,
2. for timestamp $t>k$, we decide whether to replace a $v$ with probability $k/t$,
3. if true, we replace a value (and its corresponding count) at random in the arrays $v$ and $c$ (and re-initialize the values).

In [21]:
#initialize values and counts
v = []
c = []
#keeping the true counts 
counts = {}
#choosing k timestamps
k = 10
for _ in range(k):
  # t.append(random.randrange(stream_size))
  v.append(-1)
  c.append(0)

t = [0] * k
for i in range(stream_size):
  # use Reservoir Sampling to get the timestamps
  if i < k:
    t[i] = i
  else:
    j = random.randrange(i)
    if j < k:
      t[j]=i

  #AMS approach
  #take a random value between 0 and N-1
  s = random.randrange(N)
  for j in range(k):
    if i==t[j]: #chosen timestamp
      v[j] = s
      c[j] = 1
    elif i>t[j] and s==v[j]: #after timestamp
      c[j] += 1
  #true counts (only for evaluation!)
  if s not in counts:
    counts[s] = 0
  counts[s] = counts[s]+1

#true 2nd moment
true = 0
for x in counts.keys():
  true += counts[x]*counts[x]

#2nd moment estimator
est = 0
for x in range(k):
  est += 2*c[x]-1
est = int((stream_size/k)*est)

print('Estimation of 2nd moment: %d'%est)
print('True second moment: %d'%true)

Estimation of 2nd moment: 382000
True second moment: 402024


In AMS algorithm, we randomly select a position from the stream, then calculate the number of occurrences after the selected position of the data stream. However, we have infinite streams here, so in order to randomly select a position from the stream, we used reservoir sampling algorithm here. Based on this, we implemented the AMS algorithm. The estimation and true second moment are close as shown above.